In [2]:
!pip uninstall scikit-learn -y
!pip install -U scikit-learn

Uninstalling scikit-learn-0.22.2.post1:
  Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 22.3MB 64.8MB/s 


In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

In [85]:
import os
TITANIC_PATH = ''#os.path.join("datasets", "titanic")

In [86]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [87]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

In [88]:
train_target = train_data['Survived']
train_data.drop('Survived', axis=1, inplace=True)
train_data[:5], train_target[:5]

(   PassengerId  Pclass  ... Cabin Embarked
 0            1       3  ...   NaN        S
 1            2       1  ...   C85        C
 2            3       3  ...   NaN        S
 3            4       1  ...  C123        S
 4            5       3  ...   NaN        S
 
 [5 rows x 11 columns], 0    0
 1    1
 2    1
 3    1
 4    0
 Name: Survived, dtype: int64)

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
train_data, validate_data, train_target, validate_target = train_test_split(train_data, train_target, test_size=0.2)

In [91]:
train_data.shape, validate_data.shape, test_data.shape

((712, 11), (179, 11), (418, 11))

In [92]:
(train_target == 1).sum() / train_target.size, (validate_target == 1).sum() / validate_target.size

(0.3890449438202247, 0.36312849162011174)

In [93]:
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import make_column_selector, ColumnTransformer
from collections import Counter

In [94]:
class AdditionalCollumnsAdder(BaseEstimator, TransformerMixin):
  
  def __init__(self, features):
    self.features = features
    self.titles = None

  def fit(self, X, y=None):
    counts = Counter([self.get_title(v) for v in X['Name']])
    title_categoriess = sorted(set(self.get_title(v) for v in X['Name']), key=counts.get, reverse=True)
    self.titles = {t: i for i, t in enumerate(title_categoriess)}
    # print(self.titles)
    return self

  def transform(self, X_org, y=None):
    X = X_org.copy()
    if 'NameParts' in self.features:
      X['NameParts'] = [len(v.split(' ')) for v in X['Name']]
    if 'Title' in self.features:
      if not self.titles:
        raise 'Adder has to be fitted before transofrming'
      X['Title'] = [
                    self.titles[self.get_title(v)] 
                    if self.get_title(v) in self.titles.keys() 
                    else len(self.titles) 
                    for v in X['Name']
                    ]
    if 'Comrades' in self.features:
      X['Comrades'] = [0 if v == 0 else (1 if v < 5 else 2) for v in X['SibSp'] + X['Parch']]
    if 'CabinLvl' in self.features:
      cabin = [re.sub('\d', '', v)[0] if type(v) == str else 'H' for v in X['Cabin']]
      X['CabinLvl'] = [ord(v) for v in cabin]
    if 'Social' in self.features:
      X['Social'] = [
                     0 if age < 17
                     else 1 if sex == 'Female' 
                     else 2 
                     for age, sex in zip(X['Age'], X['Sex'])
                     ]
    return pd.DataFrame(X)


  def get_title(self, v):
    return re.search('(\w+)\.', v)[0]

In [95]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
  most_frequent = None
  
  def fit(self, X, y=None):
    self.most_frequent = pd.Series([X[c].value_counts().idxmax() for c in X], 
                              index=X.columns)
    return self

  def transform(self, X_org, y=None):
    if self.most_frequent is None:
      raise 'Imputer have to be fited previously'
    X = X_org.copy()
    return X.fillna(self.most_frequent)


In [96]:
# num_cols_ex = [v for v in make_column_selector(dtype_include=np.number)(train_data) if v != 'PassengerId']
# print(num_cols_ex)

# transformer = Pipeline([
#                         ('imputer', SimpleImputer(strategy='median'))
# ])

# num_pipeline = ColumnTransformer([('numerical', transformer, num_cols_ex)], remainder='drop')

In [97]:
cat_pipeline = ColumnTransformer(
    [
    #  ('one_hot_encoding', ohe_pipeline, ['Sex', 'Embarked']),
     ('Sex_encoding', 
      Pipeline([('missing', MostFrequentImputer()), ('lab', OrdinalEncoder())]),
      ['Sex']),
     ('Embarked_encoding', 
      Pipeline([
                ('missing', MostFrequentImputer()), 
                ('lab', OrdinalEncoder(categories=[['S', 'C', 'Q']]))
                ]), 
      ['Embarked'])
     ], 
     remainder='drop')
cat_pipeline

ColumnTransformer(transformers=[('Sex_encoding',
                                 Pipeline(steps=[('missing',
                                                  MostFrequentImputer()),
                                                 ('lab', OrdinalEncoder())]),
                                 ['Sex']),
                                ('Embarked_encoding',
                                 Pipeline(steps=[('missing',
                                                  MostFrequentImputer()),
                                                 ('lab',
                                                  OrdinalEncoder(categories=[['S',
                                                                              'C',
                                                                              'Q']]))]),
                                 ['Embarked'])])

In [98]:
# ['NameParts', 'Title', 'Comrades', 'CabinLvl', 'Social']

In [109]:
s_all_cols = ['Age', 'Fare']
s_pass = ['NameParts', 'Title', 'Comrades', 'CabinLvl', 'Social', 'Pclass', 'SibSp', 'Parch']
s_add = ['NameParts', 'Title', 'Comrades', 'CabinLvl' ,'Social']

processing_pipeline = Pipeline(
    [
    ('add_collumns', AdditionalCollumnsAdder(s_add)), 
    ('transform', FeatureUnion(
        transformer_list=[
                  ('categorical_pipeline', cat_pipeline),
                  ('numerical_pipeline', make_pipeline(
                      ColumnTransformer([
                                         ('numerical', 
                                          Pipeline([
                                            ('imputer', SimpleImputer(strategy='median'))
                                          ]), 
                                          s_all_cols)
                                         ], remainder='drop'), 
                      StandardScaler())),
                      ('other', make_pipeline(
                                ColumnTransformer([
                                         ('numerical', 
                                          Pipeline([
                                            ('imputer', SimpleImputer(strategy='median'))
                                          ]), 
                                          s_pass),
                                         ], remainder='drop'), 

                      ))
                  ]))
    ])

In [110]:
q_all_cols = ['Age', 'Fare']
q_pass = ['NameParts', 'Title', 'Comrades', 'CabinLvl', 'Social', 'Pclass', 'SibSp', 'Parch']
q_add = ['NameParts', 'Title', 'Comrades', 'CabinLvl' ,'Social']

quantile_processing_pipeline = Pipeline(
    [
    ('add_collumns', AdditionalCollumnsAdder(q_add)), 
    ('transform', FeatureUnion(
        transformer_list=[
                  ('categorical_pipeline', cat_pipeline),
                  ('numerical_pipeline', make_pipeline(
                      ColumnTransformer([
                                         ('numerical', 
                                          Pipeline([
                                            ('imputer', SimpleImputer(strategy='median'))
                                          ]), 
                                          q_all_cols)
                                         ], remainder='drop'), 
                      QuantileTransformer())),
                      ('other', make_pipeline(
                                ColumnTransformer([
                                         ('numerical', 
                                          Pipeline([
                                            ('imputer', SimpleImputer(strategy='median'))
                                          ]), 
                                          q_pass),
                                         ], remainder='drop'), 

                      ))
                  ]))
    ])

In [111]:
quantile_processing_pipeline.fit_transform(train_data)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (712). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


array([[1.        , 0.        , 0.49085795, ..., 3.        , 8.        ,
        2.        ],
       [1.        , 1.        , 0.49085795, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.16455696, ..., 3.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.        , 0.80028129, ..., 3.        , 1.        ,
        5.        ],
       [1.        , 0.        , 0.90787623, ..., 3.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.99578059, ..., 1.        , 1.        ,
        1.        ]])

## Models

In [112]:
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.feature_selection import RFE

In [113]:
kfold = StratifiedKFold(n_splits=5)

### Random Forest

In [115]:
%%time
pipe = Pipeline(
    [
     ('processing', quantile_processing_pipeline),
     ('classifier', RandomForestClassifier())
    ]
)

param_grid = {
    'classifier__bootstrap': [True, False],
    'classifier__max_depth': [20, 30, 40, 60, 70, 90],
    'classifier__max_features': [0.7, 0.9, None],
    'classifier__min_samples_leaf': [0.2, 0.4, 0.6],
    'classifier__min_samples_split': [4, 6, 8, 10],
    'classifier__min_samples_leaf': [4, 6, 8],

    'classifier__n_estimators': [50, 100, 200, 300, 400, 500],
    'classifier__max_leaf_nodes': [2, 4, 8, 16, 32, 64, 128]
}

grid_1 = RandomizedSearchCV(pipe, param_grid, cv=kfold, verbose=10, n_jobs=-1, n_iter=300)

grid_1.fit(train_data, train_target)

print(grid_1.best_params_)
# {'classifier__n_estimators': 10, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 3, 'classifier__max_features': 3, 'classifier__max_depth': 100, 'classifier__bootstrap': True}
# {'classifier__n_estimators': 50, 'classifier__min_samples_split': 8, 'classifier__min_samples_leaf': 3, 'classifier__max_features': 6, 'classifier__max_depth': 110, 'classifier__bootstrap': True}
# {'classifier__n_estimators': 100, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 4, 'classifier__max_features': 9, 'classifier__max_depth': 10, 'classifier__bootstrap': True}
# {'classifier__n_estimators': 400, 'classifier__min_samples_split': 0.2, 'classifier__max_leaf_nodes': 4, 'classifier__max_features': 0.7, 'classifier__max_depth': 70, 'classifier__bootstrap': False}
# {'classifier__n_estimators': 400, 'classifier__min_samples_split': 0.2, 'classifier__max_leaf_nodes': 16, 'classifier__max_features': 0.7, 'classifier__max_depth': 60, 'classifier__bootstrap': True}
# {'classifier__n_estimators': 400, 'classifier__min_samples_split': 0.2, 'classifier__max_leaf_nodes': 8, 'classifier__max_features': 0.7, 'classifier__max_depth': 40, 'classifier__bootstrap': False}
# {'classifier__n_estimators': 200, 'classifier__min_samples_split': 0.4, 'classifier__max_leaf_nodes': 64, 'classifier__max_features': 0.7, 'classifier__max_depth': 40, 'classifier__bootstrap': False}
# {'classifier__n_estimators': 200, 'classifier__min_samples_split': 8, 'classifier__min_samples_leaf': 4, 'classifier__max_leaf_nodes': 32, 'classifier__max_features': 0.9, 'classifier__max_depth': 30, 'classifier__bootstrap': True}
# {'classifier__n_estimators': 50, 'classifier__min_samples_split': 8, 'classifier__min_samples_leaf': 6, 'classifier__max_leaf_nodes': 64, 'classifier__max_features': 0.7, 'classifier__max_depth': 20, 'classifier__bootstrap': True}
# {'classifier__n_estimators': 400, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 4, 'classifier__max_leaf_nodes': 64, 'classifier__max_features': 0.7, 'classifier__max_depth': 60, 'classifier__bootstrap': True}

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (712). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


{'classifier__n_estimators': 400, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 4, 'classifier__max_leaf_nodes': 64, 'classifier__max_features': 0.7, 'classifier__max_depth': 60, 'classifier__bootstrap': True}
CPU times: user 34.4 s, sys: 647 ms, total: 35.1 s
Wall time: 10min 44s


### XGBoost

In [116]:
"""
Pipeline(memory=None,steps=[('processing',Pipeline(memory=None,steps=
[('add_collumns',AdditionalCollumnsAdder(features=['Title','Comrades','CabinLvl'])),
('transform',FeatureUnion(n_jobs=None,transformer_list=[('categorical_pipeline',ColumnTransformer
(n_jobs=None,remainder='drop',sparse_threshold=0.3,transformer_weights=None,transformers
=[('Sex_enco...XGBClassifier(base_score=0.5,booster='gbtree',colsample_bylevel=1,
colsample_bynode=1,colsample_bytree=1.0,gamma=0.2,learning_rate=0.1,max_delta_step=0,max_depth=4
,min_child_weight=5,missing=None,n_estimators=100,n_jobs=1,nthread=None,objective='binary:logistic',
random_state=0,reg_alpha=0,
reg_lambda=1,scale_pos_weight=1,seed=None,silent=None,subsample=1.0,verbosity=1))],verbose=False)
"""

"\nPipeline(memory=None,steps=[('processing',Pipeline(memory=None,steps=\n[('add_collumns',AdditionalCollumnsAdder(features=['Title','Comrades','CabinLvl'])),\n('transform',FeatureUnion(n_jobs=None,transformer_list=[('categorical_pipeline',ColumnTransformer\n(n_jobs=None,remainder='drop',sparse_threshold=0.3,transformer_weights=None,transformers\n=[('Sex_enco...XGBClassifier(base_score=0.5,booster='gbtree',colsample_bylevel=1,\ncolsample_bynode=1,colsample_bytree=1.0,gamma=0.2,learning_rate=0.1,max_delta_step=0,max_depth=4\n,min_child_weight=5,missing=None,n_estimators=100,n_jobs=1,nthread=None,objective='binary:logistic',\nrandom_state=0,reg_alpha=0,\nreg_lambda=1,scale_pos_weight=1,seed=None,silent=None,subsample=1.0,verbosity=1))],verbose=False)\n"

In [117]:
%%time
pipe = Pipeline(
    [
     ('processing', quantile_processing_pipeline),
    #  ('pca', PCA()),
    #  ('rfe', RFE(estimator=XGBClassifier())),
     ('classifier', XGBClassifier())
    ]
)

param_grid = {
        'classifier__min_child_weight': [1, 5, 10],
        'classifier__gamma': [0.01, 0.2, 0.5, 1, 1.5, 2, 5],
        'classifier__subsample': [0.2, 0.4, 0.6, 0.8, 1.0, 5.0, 10],
        'classifier__colsample_bytree': [0.6, 0.8, 0.9,  1.0],
        'classifier__colsample_bylevel': [0.6, 0.8, 0.9, 1.0],
        'classifier__max_depth': [3, 4, 5],
        'classifier__learning_rate': [1, 0.1, 0.01]
}

grid_2 = RandomizedSearchCV(pipe, param_grid, cv=kfold, verbose=10, n_jobs=-1, n_iter=300)

grid_2.fit(train_data, train_target)

print(grid_2.best_params_)

# {'classifier__subsample': 0.6, 'classifier__min_child_weight': 1, 'classifier__max_depth': 4, 'classifier__gamma': 0.5, 'classifier__colsample_bytree': 1.0}
# {'classifier__subsample': 0.8, 'classifier__min_child_weight': 1, 'classifier__max_depth': 5, 'classifier__learning_rate': 0.1, 'classifier__gamma': 0.5, 'classifier__colsample_bytree': 1.0, 'classifier__colsample_bylevel': 0.6}
# {'classifier__subsample': 1.0, 'classifier__min_child_weight': 1, 'classifier__max_depth': 4, 'classifier__learning_rate': 0.01, 'classifier__gamma': 0.5, 'classifier__colsample_bytree': 0.9, 'classifier__colsample_bylevel': 0.9}
# {'classifier__subsample': 0.8, 'classifier__min_child_weight': 1, 'classifier__max_depth': 5, 'classifier__learning_rate': 0.1, 'classifier__gamma': 2, 'classifier__colsample_bytree': 0.9, 'classifier__colsample_bylevel': 1.0}
# {'classifier__subsample': 1.0, 'classifier__min_child_weight': 5, 'classifier__max_depth': 5, 'classifier__learning_rate': 1, 'classifier__gamma': 1, 'classifier__colsample_bytree': 0.9, 'classifier__colsample_bylevel': 1.0}
# {'classifier__subsample': 0.8, 'classifier__min_child_weight': 10, 'classifier__max_depth': 4, 'classifier__learning_rate': 1, 'classifier__gamma': 0.01, 'classifier__colsample_bytree': 1.0, 'classifier__colsample_bylevel': 0.8}
# {'classifier__subsample': 0.4, 'classifier__min_child_weight': 1, 'classifier__max_depth': 4, 'classifier__learning_rate': 0.1, 'classifier__gamma': 0.2, 'classifier__colsample_bytree': 0.9, 'classifier__colsample_bylevel': 0.6}

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
{'classifier__subsample': 0.4, 'classifier__min_child_weight': 1, 'classifier__max_depth': 4, 'classifier__learning_rate': 0.1, 'classifier__gamma': 0.2, 'classifier__colsample_bytree': 0.9, 'classifier__colsample_bylevel': 0.6}
CPU times: user 19.8 s, sys: 248 ms, total: 20.1 s
Wall time: 1min 32s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.78785581        nan        nan        nan 0.79635576 0.82298828
 0.80474737 0.82580518        nan 0.81313897 0.79210086        nan
 0.83003053 0.79352901 0.80894317 0.81734463 0.80331922 0.77939525
 0.79633606 0.80470797        nan 0.79913326 0.81034177        nan
 0.82721363        nan        nan        nan 0.81591648        nan
 0.7766473         nan        nan 0.79210086 0.76683739 0.78926426
        nan 0.76824584 0.81594603        nan        nan 0.75842608
        nan        nan 0.81174037 0.80471782 0.80617551 0.81175022
 0.82718408        nan 0.7808825         nan 0.82014183 0.81595588
 0.77807545        nan 0.81452773 0.78510785 0.81735448 0.77807545
        nan        nan        nan        nan        nan 0.82296858
        nan 0.78221215 0.7808825         nan 0.80331922 0.79214025
        nan 0.82294888 0.81877278 0.82020093 0.79493746 0

### AdaBoost

In [134]:
%%time
pipe = Pipeline(
    [
     ('processing', quantile_processing_pipeline),
    #  ('pca', PCA()),
     ('rfe', RFE(estimator=AdaBoostClassifier())),
     ('classifier', AdaBoostClassifier())
    ]
)

param_grid = {
    'classifier__n_estimators': [600, 700, 800],
    'classifier__learning_rate': [1, 0.1, 0.01, 0.001],
    'classifier__algorithm': ['SAMME', 'SAMME.R']
}

grid_3 = GridSearchCV(pipe, param_grid, cv=kfold, verbose=10, n_jobs=-1)

grid_3.fit(train_data, train_target)

print(grid_3.best_params_)
# {'classifier__n_estimators': 300, 'classifier__learning_rate': 1, 'classifier__algorithm': 'SAMME.R'}
# {'classifier__algorithm': 'SAMME', 'classifier__learning_rate': 1, 'classifier__n_estimators': 400}
# {'classifier__algorithm': 'SAMME', 'classifier__learning_rate': 1, 'classifier__n_estimators': 700}

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (712). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


{'classifier__algorithm': 'SAMME', 'classifier__learning_rate': 1, 'classifier__n_estimators': 700}
CPU times: user 4.66 s, sys: 129 ms, total: 4.79 s
Wall time: 2min 45s


## K nearest neighbours

In [ ]:
%%time
pipe = Pipeline(
    [
     ('processing', processing_pipeline),
    #  ('pca', PCA()),
    #  ('rfe', RFE(estimator=KNeighborsClassifier())),
     ('classifier', KNeighborsClassifier())
    ]
)

param_grid = {
    'classifier__n_neighbors': [5, 10, 15, 20, 25],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__leaf_size': [4, 6, 8, 16],
    'classifier__p': [1, 2, 3, 4],
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

grid_4 = RandomizedSearchCV(pipe, param_grid, cv=kfold, verbose=10, n_jobs=-1, n_iter=100)

grid_4.fit(train_data, train_target)

print(grid_4.best_params_)
# {'classifier__weights': 'uniform', 'classifier__p': 1, 'classifier__n_neighbors': 15, 'classifier__leaf_size': 4, 'classifier__algorithm': 'kd_tree'}
# {'classifier__weights': 'uniform', 'classifier__p': 1, 'classifier__n_neighbors': 20, 'classifier__leaf_size': 4, 'classifier__algorithm': 'brute'}
# {'classifier__weights': 'uniform', 'classifier__p': 1, 'classifier__n_neighbors': 15, 'classifier__leaf_size': 6, 'classifier__algorithm': 'auto'}
# {'classifier__weights': 'uniform', 'classifier__p': 1, 'classifier__n_neighbors': 5, 'classifier__leaf_size': 16, 'classifier__algorithm': 'kd_tree'}

### Naive Bias

In [ ]:
%%time
pipe = Pipeline(
    [
     ('processing', processing_pipeline),
    #  ('rfe', RFE(estimator=GaussianNB())),
     ('classifier', GaussianNB())
    ]
)

param_grid = {
    'classifier__var_smoothing': [1.9e-61, 1.95e-61, 2e-61]

}

grid_5 = RandomizedSearchCV(pipe, param_grid, cv=kfold, verbose=10, n_jobs=-1, n_iter=100)

grid_5.fit(train_data, train_target)

print(grid_5.best_params_)


### Voting

In [135]:
%%time
pipe = VotingClassifier(
    estimators=[
                ('RandomForest', grid_1.best_estimator_), 
                ('XGBoost', grid_2.best_estimator_)
                ]
)

param_grid = {
    'estimators': [
                [
                  ('XGBoost', grid_2.best_estimator_), 
                  ('AdaBoost', grid_3.best_estimator_),
                  ('KNearestNeighbours', grid_4.best_estimator_)
                ],
                [
                  ('RandomForest', grid_1.best_estimator_), 
                  ('XGBoost', grid_2.best_estimator_), 
                  ('AdaBoost', grid_3.best_estimator_),
                ],
                [
                  ('RandomForest', grid_1.best_estimator_), 
                  ('XGBoost', grid_2.best_estimator_), 
                  ('KNearestNeighbours', grid_4.best_estimator_)
                ],
                [
                  ('RandomForest', grid_1.best_estimator_), 
                  ('XGBoost', grid_2.best_estimator_), 
                  ('AdaBoost', grid_3.best_estimator_),
                  ('KNearestNeighbours', grid_4.best_estimator_)
                ],
                [
                  ('RandomForest', grid_1.best_estimator_), 
                  ('XGBoost', grid_2.best_estimator_), 
                  ('AdaBoost', grid_3.best_estimator_),
                  ('KNearestNeighbours', grid_4.best_estimator_),
                  ('NaiveBayes', grid_5.best_estimator_)
                ],
                [
                  ('XGBoost', grid_2.best_estimator_), 
                  ('KNearestNeighbours', grid_4.best_estimator_),
                  ('NaiveBayes', grid_5.best_estimator_)
                ],
                [
                  ('XGBoost', grid_2.best_estimator_), 
                  ('RandomForest', grid_1.best_estimator_), 
                  ('NaiveBayes', grid_5.best_estimator_)
                ],
                [
                  ('XGBoost', grid_2.best_estimator_), 
                  ('RandomForest', grid_1.best_estimator_)
                ]
    ],
    'voting': ['soft', 'hard']
}

grid_6 = GridSearchCV(pipe, param_grid, cv=kfold, verbose=10, n_jobs=-1)

grid_6.fit(train_data, train_target)

print(grid_6.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (712). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (712). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


{'estimators': [('XGBoost', Pipeline(steps=[('processing',
                 Pipeline(steps=[('add_collumns',
                                  AdditionalCollumnsAdder(features=['NameParts',
                                                                    'Title',
                                                                    'Comrades',
                                                                    'CabinLvl',
                                                                    'Social'])),
                                 ('transform',
                                  FeatureUnion(transformer_list=[('categorical_pipeline',
                                                                  ColumnTransformer(transformers=[('Sex_encoding',
                                                                                                   Pipeline(steps=[('missing',
                                                                                                                 

# Compare results

In [136]:
from sklearn import  metrics


models = []
models.append(('Random Forest', grid_1.best_estimator_))
models.append(('XGBoost', grid_2.best_estimator_))
models.append(('AdaBoost', grid_3.best_estimator_))
models.append(('KNearestNeighbors', grid_4.best_estimator_))
models.append(('NaiveBayes', grid_5.best_estimator_))
models.append(('Voting', grid_6.best_estimator_))

models_names = [name for name, m in models]

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(validate_target, model.predict(validate_data)) ))
    print("recall_score: {}".format( metrics.recall_score(validate_target, model.predict(validate_data)) ))
    print("f1_score: {}".format( metrics.f1_score(validate_target, model.predict(validate_data)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(validate_target, model.predict(validate_data)) ))
    precision_score.append(metrics.precision_score(validate_target, model.predict(validate_data)))
    recall_score.append(metrics.recall_score(validate_target, model.predict(validate_data)))
    f1_score.append( metrics.f1_score(validate_target, model.predict(validate_data)))
    accuracy_score.append(metrics.accuracy_score(validate_target, model.predict(validate_data)))

Random Forest
precision_score: 0.9180327868852459
recall_score: 0.8615384615384616
f1_score: 0.8888888888888888
accuracy_score: 0.9217877094972067
XGBoost
precision_score: 0.9122807017543859
recall_score: 0.8
f1_score: 0.8524590163934427
accuracy_score: 0.8994413407821229
AdaBoost
precision_score: 0.8360655737704918
recall_score: 0.7846153846153846
f1_score: 0.8095238095238095
accuracy_score: 0.8659217877094972
KNearestNeighbors
precision_score: 0.8679245283018868
recall_score: 0.7076923076923077
f1_score: 0.7796610169491526
accuracy_score: 0.8547486033519553
NaiveBayes
precision_score: 0.7301587301587301
recall_score: 0.7076923076923077
f1_score: 0.7187500000000001
accuracy_score: 0.7988826815642458
Voting
precision_score: 0.8936170212765957
recall_score: 0.6461538461538462
f1_score: 0.75
accuracy_score: 0.8435754189944135


In [137]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=models_names)
df

,Method,precision_score,recall_score,f1_score,accuracy_score
0,Random Forest,0.918033,0.861538,0.888889,0.921788
1,XGBoost,0.912281,0.800000,0.852459,0.899441
2,AdaBoost,0.836066,0.784615,0.809524,0.865922
3,KNearestNeighbors,0.867925,0.707692,0.779661,0.854749
4,NaiveBayes,0.730159,0.707692,0.718750,0.798883
5,Voting,0.893617,0.646154,0.750000,0.843575


In [138]:
model = grid_6.best_estimator_
model.fit(pd.concat((train_data, validate_data)), pd.concat((train_target, validate_target)))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (891). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (891). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


VotingClassifier(estimators=[('XGBoost',
                              Pipeline(steps=[('processing',
                                               Pipeline(steps=[('add_collumns',
                                                                AdditionalCollumnsAdder(features=['NameParts',
                                                                                                  'Title',
                                                                                                  'Comrades',
                                                                                                  'CabinLvl',
                                                                                                  'Social'])),
                                                               ('transform',
                                                                FeatureUnion(transformer_list=[('categorical_pipeline',
                                                                        

In [139]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [140]:
test_results = model.predict(test_data)
test_results

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [141]:
(test_results == 0).sum(), (test_results == 1).sum()

(273, 145)

In [142]:
output = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': test_results})
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [143]:
output.to_csv('output.csv', index=False)

In [77]:
# 0.78708